In [1]:
# 4.3
# 手写一个朴素贝叶斯的代码

In [2]:
# 这里的朴素贝叶斯用的是高斯模型，相对的还有多项式模型以及伯努利模型
# 不同的模型描述的就是贝叶斯公式中条件概率满足的不同分布
# 比如这里的高斯分布，则表示条件概率满足高斯分布的那个公式，可以用样本方差以及均值去计算概率的大小
# 一般情况下高斯模型用的最为广泛，所以手写高斯模型，sklearn里面同样有其他模型现成的库可以用

In [22]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math

In [25]:
# 数据集生成
def creat_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns = iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    # 上面的这些代码，其实就是取了我们需要的数据集，并且进行了一些修改，没什么重要的
    data = np.array(df.iloc[:100, :]) # 取整个数据集的前100个
    return data[:, :-1], data[:, -1] # 切片，和上面是一样的，label作为分类变量，也就是我们的Y

X, y = creat_data()
# 用函数来划分一下训练集以及测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_train

array([[6.4, 2.9, 4.3, 1.3],
       [5.8, 2.7, 4.1, 1. ],
       [5.1, 3.4, 1.5, 0.2],
       [5.4, 3.9, 1.3, 0.4],
       [5. , 2. , 3.5, 1. ],
       [4.6, 3.6, 1. , 0.2],
       [5.8, 2.7, 3.9, 1.2],
       [4.8, 3. , 1.4, 0.1],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [6.9, 3.1, 4.9, 1.5],
       [6.3, 2.3, 4.4, 1.3],
       [5.5, 2.4, 3.7, 1. ],
       [5.2, 3.5, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.9, 3.6, 1.4, 0.1],
       [6.2, 2.2, 4.5, 1.5],
       [5.5, 2.5, 4. , 1.3],
       [5. , 3.5, 1.6, 0.6],
       [5.7, 3.8, 1.7, 0.3],
       [4.8, 3.1, 1.6, 0.2],
       [6.6, 2.9, 4.6, 1.3],
       [4.9, 3. , 1.4, 0.2],
       [5.5, 2.6, 4.4, 1.2],
       [6.7, 3.1, 4.4, 1.4],
       [4.5, 2.3, 1.3, 0.3],
       [6.3, 2.5, 4.9, 1.5],
       [4.4, 2.9, 1.4, 0.2],
       [4.3, 3. , 1.1, 0.1],
       [5. , 3.3, 1.4, 0.2],
       [5. , 3.2, 1.2, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [5.8, 4

In [79]:
# 开始写GaussianNB的代码
class NaiveBayes:
    def __init__(self):
        self.model = None
        
    # 数学期望
    @staticmethod
    def mean(X):
        return sum(X)/float(len(X)) # 这里用float，个人认为只是引入float类型，方便后面计算
    
    # 标准差
    def stdev(self, X):
        avg = self.mean(X) # 用上面写的平均值函数算个平均值
        return math.sqrt(sum([pow(x - avg, 2) for x in X]) / float(len(X))) # 注意这里计算方差时候用的数学函数
        
    # 我们最前面提到，这里写的是高斯模型，所以我们现在可以通过高斯模型来计算概率密度函数了（也即条件概率）
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2)))) # 这个玩意其实就是高斯概率里面后面那个exp
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent
    
    # 对输入的X_train进行预处理
    def summarize(self, train_data):
        # 前面提过，在Python里面*是一个可迭代对象的解包符号
        # 这里的作用其实就是，针对训练集数据的每个特征，即每一列，将其解包后再合并起来，这样就能针对每个特征计算我们需要的均值和方差
        summarize = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
        return summarize
        
    # 接下来就是针对我们的训练数据，去得到我们需要的模型信息
    def fit(self, X, y):
        labels = list(set(y)) # 取得类别信息
        data = {label : [] for label in labels} # 这里用了一个字典，来储存分别属于不同类别（y）的训练集数据
        for f, label in zip(X, y):
            data[label].append(f) # 这一步就是往里面添加信息
        self.model = { # 同样的用一个字典来构造模型
            label : self.summarize(value) for label, value in data.items()
            # 可以发现，我们这里是利用之前写的summarize函数，将训练集进行分类并且进行预处理，从而得到了训练集每个特征的均值与方差，构建出模型
        }
        return 'gaussianNB model training is done'
        
    # 接下来，有了模型，就可以对测试集的数据计算概率从而得出测试结果了
    def calculate_probabilities(self, input_data): # 这里input其实就是我们想要预测的测试集
        probabilities = {} # 因为在预测的时候，我们是针对每一个类别得出概率，所以显然用字典来储存预测结果是十分合适的
        for label, value in self.model.items():
            probabilities[label] = 1 # 设置一个初始值
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(input_data[i], mean, stdev) # 要注意，我们设置的probabilities初始值是1
        return probabilities
    
    # 进行概率排序，得出预测类别
    def predict(self, X_test):
        label = sorted(self.calculate_probabilities(X_test).items(), key = lambda x : x[-1])[-1][0]
        # 这里的几个index，lambda表示按照概率来从排序（默认从小到大）
        # 后面-1表示取概率最大的，0表示取它的label
        return label
    
    # 算准确度
    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1
        return right / float(len(X_test)) * 100

In [80]:
# 来测试一下模型，正确答案是0.0类别，score应该是100
model = NaiveBayes()
model.fit(X_train, y_train)
print(model.predict([4.4,  3.2,  1.3,  0.2]))
model.score(X_test, y_test)

0.0


100.0

In [86]:
# 4.4
# 用sklearn库来实现gaussianNB
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
# 高斯模型、伯努利模型和多项式模型
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB()

In [87]:
clf.score(X_test, y_test)

1.0

In [88]:
clf.predict([[4.4,  3.2,  1.3,  0.2]])[0]
# 其实这里也体现出了为啥要自己会写源代码，默认的这个输出格式确实不太好看......

0.0